<img src=https://storage.googleapis.com/kaggle-datasets-images/180/384/3da2510581f9d3b902307ff8d06fe327/dataset-cover.jpg>  
## Breast Cancer Wisconsin (Diagnostic) Data Set
[Kaggle Data set](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data#data.csv): Predict whether the cancer is benign or malignant

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  #for plotting
import featuretools as ft  # featuretools for automated feature engineering

from sklearn.model_selection import KFold 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from featuretools import selection
from warnings import simplefilter  # import warnings filter

pd.options.mode.chained_assignment = None  #hide any pandas warnings
simplefilter(action='ignore', category=FutureWarning)  # ignore all future warnings
np.random.seed(123) #ensure reproducibility

### Download dataset from [Kaggle](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data/downloads/data.csv/2)

In [2]:
# Read breast cancer data set as Pandas dataframe
df_raw = pd.read_csv("data.csv")
df_raw.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


### Column Description
diagnosis: The diagnosis of breast tissues (M = malignant, B = benign)  
radius: Distances from center to points on the perimeter  
texture: Standard deviation of gray-scale values  
perimeter: Mean size of the core tumor  
smoothness: Local variation in radius lengths  
compactness: perimeter^2 / area - 1.0  
concavity: Sverity of concave portions of the contour  
concave points: Number of concave portions of the contour   
fractal_dimension: "coastline approximation" - 1  

### Clean up dataframe

In [3]:
# rearrange columns; remove coluum "Unnamed: 32"; set id as index
fixed_columns = [df_raw.columns[0]]+list(df_raw.columns[2:-1])+[df_raw.columns[1]]
df_data = df_raw[fixed_columns]

In [4]:
# Convert (M=malignant, B=benign) to (1,0)
df_data.loc[df_data.index[df_data['diagnosis']=='B'],'diagnosis'] = 0
df_data.loc[df_data.index[df_data['diagnosis']=='M'],'diagnosis'] = 1

In [5]:
df_data.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
0,842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,842517,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
2,84300903,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,84348301,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,84358402,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


### Compare the performance of machine leanring models

#### 1. Setup function for K-fold cross validation

In [6]:
def cross_validatoin(fold, model, X, y):
    """
    Perform K-fold cross validation
    compare the sensitivity, specificity, accuracy, and F1-score of input models
    """
    sensitivity=[]
    specificity=[]
    accuracy=[]
    F1scores=[]
    
    kf = KFold(n_splits=fold,shuffle=True) 
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index] 
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        cm = confusion_matrix(y_test,y_pred)
        sensitivity.append(cm[0,0]/(cm[0,0]+cm[1,0]))
        specificity.append(cm[1,1]/(cm[1,1]+cm[0,1]))
        accuracy.append(accuracy_score(y_test,y_pred))
        F1scores.append(f1_score(y_test, y_pred, pos_label=1))
        
    return [np.mean(sensitivity),np.mean(specificity),np.mean(accuracy),np.mean(F1scores)]

In [7]:
# Set the number of fold for cross validation
fold = 10

#### 2. Apply and compare machine learning models

In [8]:
# Prepare input matrix for machine learning models
feature_names = df_data.columns.tolist()[1:-1]
X = df_data[feature_names].values
y = df_data['diagnosis'].values

In [9]:
# Create dictionary to collect results
d_Model_eva = {} 

# Logistic regression
d_Model_eva['Logistic Regression'] = cross_validatoin(fold, LogisticRegression(), X, y)

# Decision tree
d_Model_eva['Decision Tree'] = cross_validatoin(fold, DecisionTreeClassifier(), X, y)

# Random forest
d_Model_eva['Random Forest'] = cross_validatoin(fold, RandomForestClassifier(), X, y)

#### 3. Determine the best model for each metric

In [10]:
# Create output dataframe
df_eva = pd.DataFrame(d_Model_eva, index=['Sensitivity','Specificity','Accuracy','F1-score'])
df_eva.round(3).T.sort_values('F1-score',ascending=False)

,Sensitivity,Specificity,Accuracy,F1-score
Random Forest,0.961,0.957,0.960,0.946
Logistic Regression,0.952,0.949,0.951,0.930
Decision Tree,0.939,0.875,0.914,0.880


### Automated Feature Engineering
Apply Automated Feature Engineering to further improve the performance of models

#### 1. Generate new features with featuretools

In [11]:
# Create new entityset
es = ft.EntitySet(id = 'breastcancer')

df_data_ft = df_data.loc[:,['id']+feature_names]

# Create an entity from the breast cancer dataframe
es = es.entity_from_dataframe(entity_id = 'breastcancer', dataframe = df_data_ft, index='id')

In [12]:
# Generate new features
df_feature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'breastcancer',
                                      trans_primitives = ['multiply_numeric','divide_by_feature'])
df_feature_matrix.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,1 / compactness_worst * radius_mean,1 / perimeter_mean * texture_se,1 / concave points_worst * concavity_se,1 / compactness_mean * smoothness_mean,1 / concave points_se * fractal_dimension_worst,1 / compactness_worst * texture_mean,1 / smoothness_worst * symmetry_se,1 / compactness_worst * texture_se,1 / compactness_mean * symmetry_se,1 / concave points_se * radius_se
id,,,,,,,,,,,,,,,,,,,,,
8670,15.46,19.48,101.70,748.9,0.10920,0.12230,0.14660,0.08087,0.1931,0.05796,...,0.270188,0.012512,234.803406,74.877426,1140.931641,0.214431,463.013980,5.315065,585.298130,192.897551
8913,12.89,13.12,81.89,515.9,0.06955,0.03729,0.02260,0.01171,0.1337,0.05581,...,0.676369,0.026037,1128.078413,385.576434,2448.995085,0.664512,642.330725,18.589330,1656.383013,1105.404766
8915,14.96,19.10,97.03,687.3,0.08992,0.09823,0.05940,0.04819,0.1879,0.05852,...,0.220610,0.010871,437.234162,113.213850,994.405079,0.172792,500.403826,3.481361,668.869208,292.825854
9047,12.94,16.17,83.18,507.6,0.09879,0.08836,0.03296,0.02390,0.1735,0.06200,...,0.394687,0.013284,739.106822,114.559552,1746.698286,0.315847,456.279316,5.643373,605.205249,938.520876
85715,13.17,18.66,85.98,534.6,0.11580,0.12310,0.12260,0.07340,0.2128,0.06777,...,0.182262,0.013014,164.863065,70.150923,698.087589,0.128638,321.233743,2.685895,466.062929,286.675468


In [13]:
print("Number of the new features: {}".format(df_feature_matrix.shape[1]-(len(feature_names)+1)))

Number of the new features: 899


#### 2. Remove highly correlated columns
Drop Highly Correlated Features, the code is adapted from [work](https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/) by Chris Albon.

In [14]:
# Define the threshold for removing correlated variables
threshold = 0.99

In [15]:
# Get correlation of each variables
corr_matrix = df_feature_matrix.corr().abs()
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,1 / compactness_worst * radius_mean,1 / perimeter_mean * texture_se,1 / concave points_worst * concavity_se,1 / compactness_mean * smoothness_mean,1 / concave points_se * fractal_dimension_worst,1 / compactness_worst * texture_mean,1 / smoothness_worst * symmetry_se,1 / compactness_worst * texture_se,1 / compactness_mean * symmetry_se,1 / concave points_se * radius_se
radius_mean,NaN,0.323782,0.997855,0.987357,0.170581,0.506124,0.676764,0.822529,0.147741,0.311631,...,0.531199,0.382589,0.127303,0.376726,0.289851,0.408353,0.081324,0.277565,0.253767,0.415550
texture_mean,NaN,NaN,0.329533,0.321086,0.023389,0.236702,0.302418,0.293464,0.071401,0.076437,...,0.186536,0.565335,0.075878,0.125724,0.179231,0.425053,0.031497,0.422089,0.158596,0.237965
perimeter_mean,NaN,NaN,NaN,0.986507,0.207278,0.556936,0.716136,0.850977,0.183027,0.261477,...,0.553105,0.391196,0.137387,0.412266,0.321657,0.435610,0.049955,0.307845,0.292736,0.435244
area_mean,NaN,NaN,NaN,NaN,0.177028,0.498502,0.685983,0.823269,0.151293,0.283110,...,0.489533,0.385393,0.121208,0.365517,0.286127,0.385177,0.047716,0.275378,0.263378,0.409364
smoothness_mean,NaN,NaN,NaN,NaN,NaN,0.659123,0.521984,0.553695,0.557775,0.584792,...,0.403112,0.175401,0.187193,0.717834,0.501909,0.386057,0.519579,0.430735,0.602741,0.439159


In [16]:
# Select columns with correlations above threshold
col_to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
print('{} columns with correlation > {}:\n-{}'.format(len(col_to_drop),threshold,"\n-".join(col_to_drop)))

231 columns with correlation > 0.99:
-perimeter_mean
-perimeter_worst
-fractal_dimension_se * perimeter_mean
-area_worst * radius_se
-concave points_worst * perimeter_mean
-area_mean * radius_worst
-perimeter_worst * symmetry_mean
-perimeter_mean * texture_worst
-compactness_worst * perimeter_worst
-perimeter_mean * perimeter_se
-area_se * perimeter_mean
-perimeter_mean * radius_mean
-area_mean * radius_se
-fractal_dimension_se * radius_mean
-concavity_se * perimeter_mean
-perimeter_worst * smoothness_mean
-radius_worst * texture_worst
-area_se * radius_worst
-concavity_se * texture_worst
-compactness_mean * radius_mean
-area_worst * radius_mean
-perimeter_mean * symmetry_worst
-fractal_dimension_worst * radius_worst
-radius_mean * radius_worst
-concavity_worst * perimeter_worst
-area_mean * perimeter_se
-concave points_mean * radius_mean
-perimeter_worst * smoothness_se
-perimeter_se * radius_worst
-radius_worst * texture_mean
-compactness_mean * perimeter_mean
-concavity_se * radius_

In [17]:
df_feature_matrix_dropcorr = df_feature_matrix.drop(columns = col_to_drop)
df_feature_matrix_dropcorr.head()

,radius_mean,texture_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,...,1 / area_se * texture_se,1 / smoothness_worst * texture_worst,1 / area_se * perimeter_se,1 / compactness_mean * smoothness_se,1 / compactness_mean * smoothness_mean,1 / concave points_se * fractal_dimension_worst,1 / smoothness_worst * symmetry_se,1 / compactness_worst * texture_se,1 / compactness_mean * symmetry_se,1 / concave points_se * radius_se
id,,,,,,,,,,,,,,,,,,,,,
8670,15.46,19.48,748.9,0.10920,0.12230,0.14660,0.08087,0.1931,0.05796,0.4743,...,0.026339,0.248781,0.006690,1310.354949,74.877426,1140.931641,463.013980,5.315065,585.298130,192.897551
8913,12.89,13.12,515.9,0.06955,0.03729,0.02260,0.01171,0.1337,0.05581,0.1532,...,0.168154,0.669198,0.070730,5668.324028,385.576434,2448.995085,642.330725,18.589330,1656.383013,1105.404766
8915,14.96,19.10,687.3,0.08992,0.09823,0.05940,0.04819,0.1879,0.05852,0.2877,...,0.042415,0.290804,0.018521,1909.262819,113.213850,994.405079,500.403826,3.481361,668.869208,292.825854
9047,12.94,16.17,507.6,0.09879,0.08836,0.03296,0.02390,0.1735,0.06200,0.1458,...,0.097269,0.370653,0.088249,3920.103277,114.559552,1746.698286,456.279316,5.643373,605.205249,938.520876
85715,13.17,18.66,534.6,0.11580,0.12310,0.12260,0.07340,0.2128,0.06777,0.2871,...,0.046142,0.200326,0.021738,1243.643118,70.150923,698.087589,321.233743,2.685895,466.062929,286.675468


#### 3. Remove columns with missing values or with too little information

In [18]:
# Remove columns with missing values
df_feature_matrix_dropcorr.replace([np.inf, -np.inf], np.nan, inplace=True)
col_without_nan = df_feature_matrix_dropcorr.columns[~df_feature_matrix_dropcorr.isna().any()]
df_feature_matrix_dropcorr_dropnan = df_feature_matrix_dropcorr[col_without_nan]

# Remove columns with too little information (less than ten distinct values)
selection.remove_low_information_features(df_feature_matrix_dropcorr_dropnan)

df_feature_matrix_dropcorr_dropnan.head()

,radius_mean,texture_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,...,1 / fractal_dimension_se * texture_worst,1 / area_se * symmetry_mean,1 / area_se * texture_se,1 / smoothness_worst * texture_worst,1 / area_se * perimeter_se,1 / compactness_mean * smoothness_se,1 / compactness_mean * smoothness_mean,1 / smoothness_worst * symmetry_se,1 / compactness_worst * texture_se,1 / compactness_mean * symmetry_se
id,,,,,,,,,,,,,,,,,,,,,
8670,15.46,19.48,748.9,0.10920,0.12230,0.14660,0.08087,0.1931,0.05796,0.4743,...,15.628419,0.107197,0.026339,0.248781,0.006690,1310.354949,74.877426,463.013980,5.315065,585.298130
8913,12.89,13.12,515.9,0.06955,0.03729,0.02260,0.01171,0.1337,0.05581,0.1532,...,30.922664,0.589861,0.168154,0.669198,0.070730,5668.324028,385.576434,642.330725,18.589330,1656.383013
8915,14.96,19.10,687.3,0.08992,0.09823,0.05940,0.04819,0.1879,0.05852,0.2877,...,13.563948,0.213992,0.042415,0.290804,0.018521,1909.262819,113.213850,500.403826,3.481361,668.869208
9047,12.94,16.17,507.6,0.09879,0.08836,0.03296,0.02390,0.1735,0.06200,0.1458,...,22.028644,0.507367,0.097269,0.370653,0.088249,3920.103277,114.559552,456.279316,5.643373,605.205249
85715,13.17,18.66,534.6,0.11580,0.12310,0.12260,0.07340,0.2128,0.06777,0.2871,...,9.821075,0.193783,0.046142,0.200326,0.021738,1243.643118,70.150923,321.233743,2.685895,466.062929


In [19]:
print("{} features are removed because they contain missing values or contain too little information.".format(df_feature_matrix_dropcorr.shape[1] - df_feature_matrix_dropcorr_dropnan.shape[1]))

94 features are removed because they contain missing values or contain too little information.


### Add diagnosis column back to the feature matrix

In [20]:
df_outcomes = df_data.loc[:,['id','diagnosis']]
df_outcomes.set_index('id',inplace=True)

df_feature_matrix_outcomes = pd.merge(df_feature_matrix_dropcorr_dropnan,df_outcomes,
                                      left_index=True, right_index=True)
df_feature_matrix_outcomes.head()

,radius_mean,texture_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,...,1 / area_se * symmetry_mean,1 / area_se * texture_se,1 / smoothness_worst * texture_worst,1 / area_se * perimeter_se,1 / compactness_mean * smoothness_se,1 / compactness_mean * smoothness_mean,1 / smoothness_worst * symmetry_se,1 / compactness_worst * texture_se,1 / compactness_mean * symmetry_se,diagnosis
id,,,,,,,,,,,,,,,,,,,,,
8670,15.46,19.48,748.9,0.10920,0.12230,0.14660,0.08087,0.1931,0.05796,0.4743,...,0.107197,0.026339,0.248781,0.006690,1310.354949,74.877426,463.013980,5.315065,585.298130,1
8913,12.89,13.12,515.9,0.06955,0.03729,0.02260,0.01171,0.1337,0.05581,0.1532,...,0.589861,0.168154,0.669198,0.070730,5668.324028,385.576434,642.330725,18.589330,1656.383013,0
8915,14.96,19.10,687.3,0.08992,0.09823,0.05940,0.04819,0.1879,0.05852,0.2877,...,0.213992,0.042415,0.290804,0.018521,1909.262819,113.213850,500.403826,3.481361,668.869208,0
9047,12.94,16.17,507.6,0.09879,0.08836,0.03296,0.02390,0.1735,0.06200,0.1458,...,0.507367,0.097269,0.370653,0.088249,3920.103277,114.559552,456.279316,5.643373,605.205249,0
85715,13.17,18.66,534.6,0.11580,0.12310,0.12260,0.07340,0.2128,0.06777,0.2871,...,0.193783,0.046142,0.200326,0.021738,1243.643118,70.150923,321.233743,2.685895,466.062929,1


In [21]:
# Prepare input matrix for machine learning models
feature_names_ft = df_feature_matrix_outcomes.columns.tolist()[1:-1]
X_ft = df_feature_matrix_outcomes[feature_names_ft].values
y_ft = df_feature_matrix_outcomes['diagnosis'].values

In [22]:
# Logistic regression
d_Model_eva['Logistic Regression (Feature Engineering)'] = cross_validatoin(fold, LogisticRegression(), X_ft, y_ft)

# Decision tree
d_Model_eva['Decision Tree (Feature Engineering)'] = cross_validatoin(fold, DecisionTreeClassifier(), X_ft, y_ft)

# Random forest
d_Model_eva['Random Forest (Feature Engineering)'] = cross_validatoin(fold, RandomForestClassifier(), X_ft, y_ft)

In [23]:
# Create output dataframe
df_eva = pd.DataFrame(d_Model_eva, index=['Sensitivity','Specificity','Accuracy','F1-score'])
df_eva.round(3).T.sort_values('F1-score',ascending=False)

,Sensitivity,Specificity,Accuracy,F1-score
Random Forest (Feature Engineering),0.956,0.974,0.961,0.948
Logistic Regression (Feature Engineering),0.965,0.953,0.960,0.947
Random Forest,0.961,0.957,0.960,0.946
Logistic Regression,0.952,0.949,0.951,0.930
Decision Tree (Feature Engineering),0.959,0.912,0.940,0.918
Decision Tree,0.939,0.875,0.914,0.880


In [24]:
# Print out conclusion
print('{}-fold cross validation shows:'.format(fold))
for index, row in df_eva.iterrows():
    print("- {} has the best {} score = {:.3f}.".format(df_eva.loc[index,:].idxmax(axis=1), index,
                                         df_eva.loc[index, df_eva.loc[index,:].idxmax(axis=1)]))

10-fold cross validation shows:
- Logistic Regression (Feature Engineering) has the best Sensitivity score = 0.965.
- Random Forest (Feature Engineering) has the best Specificity score = 0.974.
- Random Forest (Feature Engineering) has the best Accuracy score = 0.961.
- Random Forest (Feature Engineering) has the best F1-score score = 0.948.
